<a href="https://colab.research.google.com/github/KamonohashiPerry/MachineLearning/blob/master/RNN_Text_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.layers import Dense, Activation, SimpleRNN
from keras.models import Sequential
import numpy as np

import codecs

In [3]:

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 145118 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.18-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [4]:
# drive mean root directory of  google drive
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive/"Colab Notebooks"/NLP

alice_in_wonderland.txt  Untitled0.ipynb


In [0]:
INPUT_FILE = "drive/Colab Notebooks/NLP/alice_in_wonderland.txt"
with codecs.open(INPUT_FILE, "r", encoding="utf-8") as f:
  lines = [line.strip().lower() for line in f if len(line) != 0 ]
  text = " ".join(lines)

In [0]:
chars = set(text)
nb_chars = len(chars)
char2index = dict((c, i) for i, c in enumerate(chars))
index2char = dict((i, c) for i, c in enumerate(chars))

In [0]:
SEQLEN = 10
STEP = 1

input_chars = []
label_chars = []
for i in range(0, len(text) - SEQLEN, STEP ):
  input_chars.append(text[i:i + SEQLEN])
  label_chars.append(text[i + SEQLEN])

In [0]:
# input_chars

In [0]:
# label_chars

In [0]:
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)

for i, input_char in enumerate(input_chars):
  for j, ch in enumerate(input_char):
    X[i, j, char2index[ch]] = 1
  y[i, char2index[label_chars[i]]] = 1

In [24]:
X.shape

(163816, 10, 63)

In [25]:
y.shape

(163816, 63)

In [27]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False,
                    input_shape=(SEQLEN, nb_chars),
                    unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [29]:
for iteration in range(NUM_ITERATIONS):
  print("=" * 50)
  print("Iteration #: %d" % (iteration))
  model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

  test_idx = np.random.randint(len(input_chars))
  test_chars = input_chars[test_idx]
  print("Generating from seed: %s" % (test_chars))
  print(test_chars, end="")

  for i in range(NUM_PREDS_PER_EPOCH):
    Xtest = np.zeros((1, SEQLEN, nb_chars))
    for j, ch in enumerate(test_chars):
      Xtest[0, j, char2index[ch]] = 1
    pred = model.predict(Xtest, verbose=0)[0]
    ypred = index2char[np.argmax(pred)]
    print(ypred, end="")

    test_chars = test_chars[1:] + ypred
  print()

Iteration #: 0
Epoch 1/1
163816/163816 [==============================] - 5s 28us/step - loss: 1.5067
Generating from seed: of feet on
of feet on the torter and the mouse to her alice to the project gutenberg-tm electronic works to ear project g
Iteration #: 1
Epoch 1/1
163816/163816 [==============================] - 5s 28us/step - loss: 1.4931
Generating from seed:  would be 
 would be of the dide her for the door as a little of the dide her for the door as a little of the dide her fo
Iteration #: 2
Epoch 1/1
163816/163816 [==============================] - 5s 29us/step - loss: 1.4809
Generating from seed: te of mind
te of mind a nother a tone of the mock turtle sormouse so she was so she was so she was so she was so she was 
Iteration #: 3
Epoch 1/1
163816/163816 [==============================] - 5s 28us/step - loss: 1.4699
Generating from seed: e queen wi
e queen with a seres of the word as it was and here the project gutenberg-tm electronic work on the read a men
Iteration #: 4
E